In [ ]:
# This Python 3 environ|ment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import keras
import tensorflow as tf
# import the necessary packages
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Dense, Input
from keras import optimizers
from keras.optimizers import Adam
from keras.optimizers import Adamax
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
import matplotlib.pyplot as pyplot
from PIL import Image
from zipfile import ZipFile
import pydicom
import scipy.ndimage


# ***Image-read***

In [ ]:
train_dir=os.path.join("/kaggle/input/eye-diagram-1000" )

In [ ]:
#from skimage import color
#from skimage import io

#img = io.imread('/kaggle/input/eye-diagram-1000')
#imgGray = color.rgb2gray(img)

In [ ]:
train_data_gen= ImageDataGenerator(rescale=1./255,validation_split=0.1)
train_generator = train_data_gen.flow_from_directory(train_dir,target_size=(256,256),batch_size=10,shuffle=False,color_mode="grayscale",class_mode= 'input', subset='training')
val_generator = train_data_gen.flow_from_directory(train_dir,target_size=(256,256),batch_size=10,shuffle=False,color_mode="grayscale",class_mode= 'input', subset='validation')

# ***ENCODER***

In [ ]:
#encoder
filters=(32, 64, 128)
chanDim = -1
latentDim=64
# define the input to the encoder
inputs = Input(shape=(256,256,1))
x = inputs
# loop over the number of filters
for f in filters:
	# apply a CONV => RELU => BN operation
	x = Conv2D(f, (3, 3), strides=2, padding="same")(x)
	x = LeakyReLU(alpha=0.2)(x)
	x = BatchNormalization(axis=chanDim)(x)
# flatten the network and then construct our latent vector
volumeSize = K.int_shape(x)
x = Flatten()(x)
latent = Dense(latentDim)(x)
# build the encoder model
encoder = Model(inputs, latent, name="encoder")

In [ ]:
encoder.summary()

# ***DECODER***

In [ ]:
#decoder
# output of the encoder as its inputs
latentInputs = Input(shape=(latentDim,))
x = Dense(np.prod(volumeSize[1:]))(latentInputs)
x = Reshape((volumeSize[1], volumeSize[2], volumeSize[3]))(x)

for f in filters[::-1]:
	# apply a CONV_TRANSPOSE => RELU => BN operation
	x = Conv2DTranspose(f, (3, 3), strides=2,padding="same")(x)
	x = LeakyReLU(alpha=0.2)(x)
	x = BatchNormalization(axis=chanDim)(x)

In [ ]:
# apply a single CONV_TRANSPOSE layer used to recover the
# original depth of the image
x = Conv2DTranspose(1, (3, 3), padding="same")(x)
outputs = Activation("sigmoid")(x)
# build the decoder model
decoder = Model(latentInputs, outputs, name="decoder")
# our autoencoder is the encoder + decoder
autoencoder = Model(inputs, decoder(encoder(inputs)),name="autoencoder")

In [ ]:
decoder.summary()

In [ ]:
autoencoder.summary()

In [ ]:
autoencoder.compile(loss='mean_squared_error', optimizer = Adamax(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07))


In [ ]:
#x= train_x(what we input), y= train_x(what is desired output) same for validation 
trained_model = autoencoder.fit(train_generator, batch_size=10, epochs=1, verbose=1,validation_data=val_generator)

#  ***RESULTS***

In [ ]:
pyplot.plot(trained_model.history['loss'], label='train')
pyplot.plot(trained_model.history['val_loss'], label='val')
pyplot.legend()
pyplot.savefig('gray_eye_auto.png')
pyplot.show()

In [ ]:
decoded_output = autoencoder.predict(train_generator)*255
np.shape(decoded_output)
#print(decoded_output[99,210,50,:])
autoencoder.evaluate(val_generator)

In [ ]:
import cv2
#pyplot.imshow(decoded_output[120,:,:], cmap= 's')
input_image=pyplot.imread("../input/eye-diagram-1000/CNN_IL_Eye/FFE_Mat_2sps_Taps_5_Tag_114.jpg")
pyplot.savefig('plot_eye_101.jpg')
#pyplot.imshow(cv2.cvtColor(decoded_output[,:,:,:], cv2.COLOR_BGR2RGB))
pyplot.imshow(decoded_output[18,:,:,:])
pyplot.show()
pyplot.savefig('plot_eye_pred_101.jpg')
pyplot.imshow(input_image)
#print(input_image[2,120,:,])


In [ ]:
img = cv2.imread('../input/eye-diagram-1000/CNN_IL_Eye/FFE_Mat_2sps_Taps_5_Tag_103.jpg')
train_data_gen_1= ImageDataGenerator(rescale=1./255)
train_generator_1 = train_data_gen.flow('../input/eye-diagram-1000/CNN_IL_Eye/FFE_Mat_2sps_Taps_5_Tag_103.jpg',target_size=(256,256),batch_size=10,seed=42,shuffle=False,color_mode="grayscale")
decoded_output_1 = autoencoder.predict(train_generator_1)*255
pyplot.imshow(decoded_output[0,:,:,:])
pyplot.show()

In [ ]:
encoder.save('encoder_eye_32_3_gray.h5')